In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
#needed for Jupyter Notebook, if want plots to show inline
%matplotlib inline 
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [2]:
import glob
import re
#plt.style.available
plt.style.use('seaborn-colorblind')

In [3]:
from collections import Counter
from scipy.spatial.distance import pdist, squareform

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
# %whos


## Loading data

In [6]:
def read2000sMatches(dirname):
    allFiles = glob.glob(dirname + "/atp_matches_2" + "*.csv")
    matches = pd.DataFrame()
    list_ = list()
    for filen in allFiles:
        df = pd.read_csv(filen,
                         index_col=None,
                         header=0,
                         parse_dates=[0])
                         #date_parser=lambda t:parse(t))
        list_.append(df)
    matches = pd.concat(list_)
    return matches

def read1900sMatches(dirname):
    allFiles = glob.glob(dirname + "/atp_matches_1" + "*.csv")
    matches = pd.DataFrame()
    list_ = list()
    for filen in allFiles:
        df = pd.read_csv(filen,
                         index_col=None,
                         header=0,
                         parse_dates=[0])
                         #date_parser=lambda t:parse(t))
        list_.append(df)
    matches = pd.concat(list_)
    return matches

In [7]:
matches2000s=read2000sMatches('tennis/')
matches1900s=read1900sMatches('tennis/')
matches_SackmannAll = pd.concat([matches1900s,matches2000s])

## Take only data with detailed stats

In [8]:
df_stats = matches_SackmannAll[~pd.isnull(matches_SackmannAll['w_ace'])]


In [9]:
df_win = df_stats[['tourney_id', 'match_num','tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date',  'score', 'best_of',
       'round', 'minutes','winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points','w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced']]
df_loss = df_stats[['tourney_id','match_num','tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date',  'score', 'best_of',
       'round', 'minutes','loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points','l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced']]

### Make 'win' and 'loss' columns compatible

#### Situation before

In [10]:
df_win_loss = pd.concat([df_win,df_loss],axis=1)
# each row has basic info repeated
# each row has two 'target'

#### Making compatible

In [11]:
df_win.columns = [s.replace("winner_", "") for s in df_win.columns]
df_win.columns = [s.replace("w_", "") for s in df_win.columns]
df_loss.columns = [s.replace("loser_", "") for s in df_win.columns]
df_loss.columns = [s.replace("l_", "") for s in df_win.columns]

df_win['target']=1
df_loss['target']=0

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [12]:
df_win_loss = pd.concat([df_win,df_loss])


#### Fill missing values

In [13]:
#seed, entry (str),have 

# 2 scores missing,~5000 missing minutes, ~35 missing 'hand', 
# ~7000 missing heights, ~100 missing age





## fill method of entry for most players with 'standard'; mostly missing values
df_win_loss['entry'] = df_win_loss['entry'].fillna('standard')
## fill unseed players with -9999; mostly missing values
## better way might be max(33,rank) ## 33 is the minumum non-seeded value
df_win_loss['seed'] = df_win_loss['seed'].fillna(9999)
df_win_loss['seed'] = df_win_loss['seed'].astype(int)

#rank, and rank_points have ~4000 missign out of 162000
df_win_loss[['rank','rank_points']] = df_win_loss.groupby('name')[['rank','rank_points']]\
                                        .transform(lambda x: x.fillna(x.mean()))
#rank, and rank_points have some more missing values: set to maximum ranking and minimum rank points
df_win_loss['rank'] = df_win_loss['rank']\
                                        .transform(lambda x: x.fillna(x.max()))
df_win_loss['rank_points'] = df_win_loss['rank_points']\
                                        .transform(lambda x: x.fillna(x.min()))


#### check effectiveness of filling

In [14]:
#length 0, good
len(df_win_loss.loc[pd.isnull(df_win_loss['entry']),'entry'])
#length 0, good
len(df_win_loss.loc[pd.isnull(df_win_loss['seed']),'seed'])
# length 179
len(df_win_loss.loc[pd.isnull(df_win_loss['rank']),'rank'])
# length 179
len(df_win_loss.loc[pd.isnull(df_win_loss['rank_points']),'rank_points'])

# missing values for 'rank','rank_points' are either unusual 'entry' (Q,LL,WC) or are Davis cup matches
# .. 'tourney_id' contains '-DC-' and 'tourney_name' contains 'Davis Cup'
df_win_loss.loc[pd.isnull(df_win_loss['rank']),:].head(100)
# df_win_loss.loc[pd.isnull(df_win_loss['rank']),:].tail(100)

,tourney_id,match_num,tourney_name,surface,drasize,tourney_level,tourney_date,score,best_of,round,minutes,id,seed,entry,name,hand,ht,ioc,age,rank,rank_points,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,target


In [15]:
df_win_loss['hand'].unique()#'R', 'L', 'U', nan
df_win_loss['hand'].value_counts()

R    138155
L     23661
U       712
Name: hand, dtype: int64

#### more filling of missing values

In [16]:
### for this 1st epicycle: don't want score, 
# scores missing,, , 
 

## set 'hand' to U ('unknown') ~35 missing 'hand'
df_win_loss['hand'] = df_win_loss['hand'].fillna('U')

In [17]:
# set missing height, age  to average for all people ~7000 missing heights, ~100 missing age
df_win_loss[['age','ht']] = df_win_loss[['age','ht']]\
                                        .transform(lambda x: x.fillna(x.mean()))

In [18]:
## set missing minutes to average age of all ~5000 missing minutes
df_win_loss['minutes'] = df_win_loss['minutes']\
                                        .transform(lambda x: x.fillna(x.mean()))

In [19]:
(df_win_loss['age']).value_counts() #87 (most) occurrences of 23.7919
len(df_win_loss.loc[pd.isnull(df_win_loss['age']),'age']) # 0 good
(df_win_loss['ht']).value_counts() #25899 (most) occureences of 185 (cm)
len(df_win_loss.loc[pd.isnull(df_win_loss['ht']),'ht']) # 0 good

0

In [20]:
df_win_loss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162562 entries, 15 to 2823
Data columns (total 31 columns):
tourney_id       162562 non-null object
match_num        162562 non-null int64
tourney_name     162562 non-null object
surface          162562 non-null object
drasize          162562 non-null int64
tourney_level    162562 non-null object
tourney_date     162562 non-null int64
score            162560 non-null object
best_of          162562 non-null int64
round            162562 non-null object
minutes          162562 non-null float64
id               162562 non-null int64
seed             162562 non-null int64
entry            162562 non-null object
name             162562 non-null object
hand             162562 non-null object
ht               162562 non-null float64
ioc              162562 non-null object
age              162562 non-null float64
rank             162562 non-null float64
rank_points      162562 non-null float64
ace              162562 non-null float64
df        

In [21]:
df_win_loss['hand'].value_counts() #R    138155 L     23661 U       746

R    138155
L     23661
U       746
Name: hand, dtype: int64

df_win_loss['rank'].value_counts()
max(df_win_loss['rank'].unique()) #2159!
max(df_loss['rank']) # checks: rank 2159 does exist

In [22]:
df_win_loss.columns
type(df_win_loss.info()) # none type
# df_win_loss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162562 entries, 15 to 2823
Data columns (total 31 columns):
tourney_id       162562 non-null object
match_num        162562 non-null int64
tourney_name     162562 non-null object
surface          162562 non-null object
drasize          162562 non-null int64
tourney_level    162562 non-null object
tourney_date     162562 non-null int64
score            162560 non-null object
best_of          162562 non-null int64
round            162562 non-null object
minutes          162562 non-null float64
id               162562 non-null int64
seed             162562 non-null int64
entry            162562 non-null object
name             162562 non-null object
hand             162562 non-null object
ht               162562 non-null float64
ioc              162562 non-null object
age              162562 non-null float64
rank             162562 non-null float64
rank_points      162562 non-null float64
ace              162562 non-null float64
df        

NoneType

Index(['tourney_id', 'match_num', 'tourney_name', 'surface', 'drasize',
       'tourney_level', 'tourney_date', 'score', 'best_of', 'round', 'minutes',
       'id', 'seed', 'entry', 'name', 'hand', 'ht', 'ioc', 'age', 'rank',
       'rank_points', 'ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon',
       'SvGms', 'bpSaved', 'bpFaced', 'target'],
      dtype='object')

In [23]:
df_win_loss['surface'].unique() #['Hard', 'Carpet', 'Clay', 'Grass', 'None']
df_win_loss['surface'].value_counts()
df_win_loss = pd.concat([df_win_loss,\
                         pd.get_dummies(df_win_loss['surface'])],axis=1)

In [24]:
df_win_loss = pd.concat([df_win_loss,\
                         pd.get_dummies(df_win_loss['hand'])],axis=1)

In [25]:
df_win_loss.columns

Index(['tourney_id', 'match_num', 'tourney_name', 'surface', 'drasize',
       'tourney_level', 'tourney_date', 'score', 'best_of', 'round', 'minutes',
       'id', 'seed', 'entry', 'name', 'hand', 'ht', 'ioc', 'age', 'rank',
       'rank_points', 'ace', 'df', 'svpt', '1stIn', '1stWon', '2ndWon',
       'SvGms', 'bpSaved', 'bpFaced', 'target', 'Carpet', 'Clay', 'Grass',
       'Hard', 'None', 'L', 'R', 'U'],
      dtype='object')

* Hard      80028 
* Clay      54282 
* Grass     16366 
* Carpet    11736 
* None        150 

Name: surface, dtype: int64

In [26]:
df_wl_surf = df_win_loss.drop('surface', axis=1,inplace=True)

In [27]:
df_win_loss.drop('hand', axis=1,inplace=True)

In [28]:
df_win_loss.info() # all data full 162562 rows non-null except 2 null scores
df_win_loss.columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162562 entries, 15 to 2823
Data columns (total 37 columns):
tourney_id       162562 non-null object
match_num        162562 non-null int64
tourney_name     162562 non-null object
drasize          162562 non-null int64
tourney_level    162562 non-null object
tourney_date     162562 non-null int64
score            162560 non-null object
best_of          162562 non-null int64
round            162562 non-null object
minutes          162562 non-null float64
id               162562 non-null int64
seed             162562 non-null int64
entry            162562 non-null object
name             162562 non-null object
ht               162562 non-null float64
ioc              162562 non-null object
age              162562 non-null float64
rank             162562 non-null float64
rank_points      162562 non-null float64
ace              162562 non-null float64
df               162562 non-null float64
svpt             162562 non-null float64
1stIn   

Index(['tourney_id', 'match_num', 'tourney_name', 'drasize', 'tourney_level',
       'tourney_date', 'score', 'best_of', 'round', 'minutes', 'id', 'seed',
       'entry', 'name', 'ht', 'ioc', 'age', 'rank', 'rank_points', 'ace', 'df',
       'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved', 'bpFaced',
       'target', 'Carpet', 'Clay', 'Grass', 'Hard', 'None', 'L', 'R', 'U'],
      dtype='object')

In [ ]:
df_win_loss['drasize'].value_counts()

32     72612
128    28192
56     20302
48     12666
28     10258
64      9496
96      6964
4       1322
8        630
9         60
16        30
10        30
Name: drasize, dtype: int64

In [29]:
df_pred_cols=df_win_loss[['drasize', 'tourney_level',
       'tourney_date', 'score', 'best_of', 'round', 'minutes', 'id', 'seed',
       'entry', 'name', 'ht', 'ioc', 'age', 'rank', 'rank_points', 'ace', 'df',
       'svpt', '1stIn', '1stWon', '2ndWon', 'SvGms', 'bpSaved', 'bpFaced',
       'target', 'Carpet', 'Clay', 'Grass', 'Hard', 'None', 'L', 'R', 'U']]

KeyError: "[''] not in index"

# Remedials

* For scatter matrix plots, plot **differences** of eg. height, age 
* For comparing wins and losses, 

# Problems

* *used all the data to get average rank: not terrible but leverages future data* : used for filling missing 'rank' and 'rank_points' (?)

* *mising ranks (after filling with plyaer average) filled with maximum rank*: is that ok?
* *mising age ht (height) minutes as average of all cols*: problem--players with higher ranks will be represented more
* *Kudos Sam: Negative data found in l_bpSaved: only two values

# Directions

* *Dastan: using S3 from which EC2 pulls -- would be a good skill set*
* *Joe: can open up a few EC2s*--model1, -model2, --grid search 
* *am **predicting** matches between players, so have to **train on** matches between plyaers*
* *Joe suggested to Sam: Do **lasso regression** and **use** features with **large betas** for logistic or can do decision tree/RandomForest and get most important features

In [ ]:
lgstc_reg_model=LogisticRegression()